<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on new_embedding_keras.ipynb

references:

https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go

https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility

https://www.kaggle.com/canming/ensemble-mean-iii-64-36

https://www.kaggle.com/tks0123456789/pme-ema-6-x-8-epochs

https://www.kaggle.com/bkkaggle/pytorch-determinism-test

Please verify your config values.

In [1]:
seed = 31
n_splits = 5
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'

In [6]:
import os
import gc
import random

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
def seed_torch(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [8]:
seed_torch()

In [9]:
import os

train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [10]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(test_path + '.zip')

print('train shape:', train_df.shape)
print('test shape:', test_df.shape)

train shape: (1306122, 3)
test shape: (1306122, 3)


In [11]:
import zipfile
from gensim.models import KeyedVectors

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:
        print("Found embeddings as a zip file")

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [12]:
%%time
glove_embedding = load_embedding(glove)

Found embeddings as a zip file
CPU times: user 3min 11s, sys: 4.09 s, total: 3min 15s
Wall time: 3min 17s


We use all tokens from both train and test data for our vocabulary.

Tokenizer : keras

In [14]:
puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [15]:
from keras.preprocessing.text import Tokenizer

to_exclude = ''
to_tokenize = puncts

tokenizer = Tokenizer(filters=to_exclude, lower=False)

In [16]:
from sklearn.model_selection import train_test_split, KFold

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

train_df, val_df = train_test_split(train_df, test_size=0.1)

train_idx, val_idx = list(kfold.split(train_df))[0]
train_df, val_df = train_df.iloc[train_idx], train_df.iloc[val_idx]

train_text = train_df["question_text"].fillna("_na_")
val_text = val_df["question_text"].fillna("_na_")
test_text = test_df["question_text"].fillna("_na_")

train_y = train_df['target'].values
val_y = val_df['target'].values

In [17]:
import re

train_text = train_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
val_text = val_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
test_text = test_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values

In [18]:
text = list(train_text) + list(val_text) + list(test_text)

In [19]:
%%time
tokenizer.fit_on_texts(text)

CPU times: user 31.2 s, sys: 115 ms, total: 31.3 s
Wall time: 31.2 s


In [20]:
word_index = tokenizer.word_index

In [21]:
%%time
train_X = tokenizer.texts_to_sequences(train_text)
val_X = tokenizer.texts_to_sequences(val_text)
test_X = tokenizer.texts_to_sequences(test_text)

maxlen = max({len(seq) for seq in train_X} | {len(seq) for seq in val_X} | {len(seq) for seq in test_X})

CPU times: user 26.9 s, sys: 306 ms, total: 27.2 s
Wall time: 27.1 s


In [22]:
from keras.preprocessing.sequence import pad_sequences

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [23]:
glove_emb_mean, glove_emb_std = -0.005838499, 0.48782197

In [24]:
embed_size = 300 # how big is each word vector
max_features = len(word_index) # how many unique words to use (i.e num rows in embedding vector)

Without any effort

In [ ]:
show_inital_emb_matrix = False # set False if you do not want to see the intial embedding matrix

In [ ]:
if show_inital_emb_matrix:
    glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
    glove_oov = {}
    for word, i in tqdm(word_index.items()):
        if word in glove_embedding:
            glove_emb_matrix[i] = glove_embedding[word]
        
        else:
            glove_oov[word] = id

    print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))

else:
    print('percentage of oov of glove: 41.39%')

percentage of oov of glove: 41.39%


Let's make an improved embedding matrix.

In [25]:
import nltk
nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
s = PorterStemmer()
l = LancasterStemmer()
n = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [26]:
def is_in_emb(word, i, embedding, emb_matrix):

    if word in embedding:
        emb_matrix[i] = embedding[word]
        return True
    
    tmp = word.lower()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = word.upper()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = word.capitalize()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = s.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = l.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = n.lemmatize(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    return False

In [27]:
glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
glove_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, glove_embedding, glove_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue

    else:
        glove_oov[word] = i

print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))


percentage of oov of glove: 14.11%


In [28]:
del glove_embedding
gc.collect()

0

41.39% -> 14.11%

Wiki

In [ ]:
%%time
wiki_embedding = load_embedding(wiki)

Found embeddings as a zip file
CPU times: user 1min 17s, sys: 2.04 s, total: 1min 20s
Wall time: 1min 20s


In [ ]:
wiki_emb_mean, wiki_emb_std = -0.0033469985, 0.109855495

In [ ]:
if show_inital_emb_matrix:
    wiki_emb_matrix = np.random.normal(wiki_emb_mean, wiki_emb_std, (max_features, embed_size))
    wiki_oov = {}
    for word, i in tqdm(word_index.items()):
        if word in wiki_embedding:
            wiki_emb_matrix[i] = wiki_embedding[word]
        
        else:
            wiki_oov[word] = id

    print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))

else:
    print('percentage of oov of wiki: 46.91%')

percentage of oov of wiki: 46.91%


In [ ]:
wiki_emb_matrix = np.random.normal(wiki_emb_mean, wiki_emb_std, (max_features, embed_size))
wiki_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, wiki_embedding, wiki_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, wiki_embedding, wiki_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, wiki_embedding, wiki_emb_matrix): continue

    else:
        wiki_oov[word] = i

print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))


percentage of oov of wiki: 17.85%


In [ ]:
del wiki_embedding
gc.collect()

102

google

In [ ]:
%%time
google_embedding = load_embedding(google)

Found embeddings as a zip file
CPU times: user 1min 42s, sys: 3.1 s, total: 1min 45s
Wall time: 1min 45s


In [ ]:
google_emb_mean, google_emb_std = -0.003527845, 0.13315111

In [ ]:
google_emb_matrix = np.random.normal(google_emb_mean, google_emb_std, (max_features, embed_size))
google_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, google_embedding, google_emb_matrix): continue

    tmp = re.sub('[0-9]{5,}', '#####', word)
    tmp = re.sub('[0-9]{4}', '####', tmp)
    tmp = re.sub('[0-9]{3}', '###', tmp)
    tmp = re.sub('[0-9]{2}', '##', tmp)
    if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

    # tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, google_embedding, google_emb_matrix): continue

    else:
        google_oov[word] = i

print('percentage of oov of google: {:.2f}%'.format(len(google_oov) / max_features * 100))


percentage of oov of google: 21.32%


In [ ]:
del google_embedding
gc.collect()

369

para

In [29]:
%%time
para_embedding = load_embedding(para)

Found embeddings as a zip file
CPU times: user 2min 34s, sys: 2.63 s, total: 2min 37s
Wall time: 2min 38s


In [30]:
para_emb_mean, para_emb_std = -0.0053248387, 0.49346521

In [31]:
para_emb_matrix = np.random.normal(para_emb_mean, para_emb_std, (max_features, embed_size))
para_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, para_embedding, para_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, para_embedding, para_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, para_embedding, para_emb_matrix): continue

    else:
        para_oov[word] = i

print('percentage of oov of para: {:.2f}%'.format(len(para_oov) / max_features * 100))


percentage of oov of para: 13.38%


In [32]:
del para_embedding
gc.collect()

208

Embedding averaging

https://medium.com/@ph_singer/1st-place-in-kaggle-quora-insincere-questions-classification-competition-520616d39938

In [33]:
emb_matrix = np.mean((0.7 * glove_emb_matrix, 0.3 * para_emb_matrix), axis=0)

https://www.kaggle.com/leighplt/glove-wiki-gnews-full-set

In [34]:
class RecNN(nn.Module):
    def __init__(self, embs_vocab, hidden_size, layers=1, dropout=0., bidirectional=False):
        super().__init__()

        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.num_layers = layers
        self.emb = nn.Embedding(embs_vocab.size(0), embs_vocab.size(1))
        self.emb.weight.data.copy_(embs_vocab) # load pretrained vectors
        self.emb.weight.requires_grad = False # make embedding non trainable
        
        self.line = nn.Conv1d(300, 300, 1, bias=True) # Linear transformation of embedding vectors
        
        self.lstm = nn.LSTM(embs_vocab.size(1), self.hidden_size,
                            num_layers=layers, bidirectional=bidirectional, dropout=dropout)
        
        self.gru = nn.GRU(embs_vocab.size(1), self.hidden_size,
                            num_layers=layers, bidirectional=bidirectional, dropout=dropout)
        
        self.out = nn.Linear(self.hidden_size*(bidirectional + 1), 32)
        self.last = nn.Linear(32, 1)
                
    def forward(self, x):
        
        embs = self.emb(x)
        lstm, (h, c) = self.lstm(embs)
        
        x = F.relu(self.line(embs.permute(1,2,0)), inplace=True).permute(2,0,1)
        gru, h = self.gru(x, h)
        lstm = lstm + gru
        
        lstm, _ = lstm.max(dim=0, keepdim=False) 
        out = self.out(lstm)
        out = self.last(F.relu(out)).squeeze()
        return out

https://www.kaggle.com/peining/third-place-model-for-toxic-comments-in-pytorch

In [106]:
loss_fn = torch.nn.CrossEntropyLoss().cuda()

https://www.kaggle.com/lhohoz/quora-lstm-gru-pytorch

In [84]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix):
        super(NeuralNet, self).__init__()
        
        hidden_size = 60
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        # self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        # self.bn = nn.BatchNorm1d(16, momentum=0.5)
        # self.lstm_attention = Attention(hidden_size*2, maxlen)
        # self.gru_attention = Attention(hidden_size*2, maxlen)
        self.linear = nn.Linear(480, 16)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(512, 1)
        # self.gaussianNoise = GaussianNoise(0.1)
        
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))
        
        h_lstm, _ = self.lstm(h_embedding)
        # h_gru, _ = self.gru(h_lstm)
        
        # h_lstm_atten = self.lstm_attention(h_lstm)
        # h_gru_atten = self.gru_attention(h_gru)
        
        # avg_pool = torch.mean(h_gru, 1)
        # max_pool, _ = torch.max(h_gru, 1)
        
        # conc = torch.cat((h_lstm, h_gru, avg_pool, max_pool), 1)
        # conc = self.relu(self.linear(conc))
        # conc = self.bn(conc)
        # conc = self.dropout(conc)
        # conc = self.gaussianNoise(conc)
        print(h_lstm.shape)
        out = self.out(h_lstm)
        
        return out

https://wikidocs.net/66747

In [97]:
class LSTM(nn.Module):
    def __init__(self, emb_matrix):
        super(LSTM, self).__init__()

        hidden_dim = 60
        max_features, embed_size = emb_matrix.shape

        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(emb_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = nn.Dropout2d(0.1)

        self.lstm = nn.LSTM(embed_size, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)
        self.dropout = nn.Dropout(p=0.25)
    
    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions


In [89]:
emb_matrix.shape

(298919, 300)

In [91]:
max_features

298919

In [44]:
ttrain_X = torch.tensor(train_X).cuda()

In [45]:
ttrain_y = torch.tensor(train_y).cuda()

In [46]:
train = torch.utils.data.TensorDataset(ttrain_X, ttrain_y)

In [123]:
batch_size = 64

In [124]:
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

In [125]:
model = LSTM(emb_matrix)

In [126]:
model.cuda()

LSTM(
  (embedding): Embedding(298919, 300)
  (embedding_dropout): Dropout2d(p=0.1, inplace=False)
  (lstm): LSTM(300, 60, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=120, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [127]:
tval_X = torch.tensor(val_X).cuda()
tval_y = torch.tensor(val_y).cuda()
val = torch.utils.data.TensorDataset(tval_X, tval_y)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False)

In [128]:
step_size = 300
base_lr, max_lr = 0.001, 0.003
optimizer = torch.optim.Adam(model.parameters())

In [129]:
best_score = -np.inf

In [131]:
for epoch in range(2):
    model.train()
    avg_loss = 0.
    for x_batch, y_batch in tqdm(train_loader):
        y_pred = model(x_batch)
        # print(y_pred.shape)
        y_batch = y_batch.view(-1, 1)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
        print(avg_loss)







In [132]:
avg_loss

0.23436437669557864

In [135]:
model.eval()

LSTM(
  (embedding): Embedding(298919, 300)
  (embedding_dropout): Dropout2d(p=0.1, inplace=False)
  (lstm): LSTM(300, 60, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=120, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [ ]:
# del glove_embedding, para_embdding, google_embedding, wiki_embedding
# gc.collect()

In [ ]:
emb_matrix.shape

(298919, 300)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


https://stackoverflow.com/questions/50747947/embedding-in-pytorch

https://www.kaggle.com/tks0123456789/pme-ema-6-x-8-epochs

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [ ]:
train_X

array([[     0,      0,      0, ...,     77,  11994,      1],
       [     0,      0,      0, ...,   3422,  23617,      1],
       [     0,      0,      0, ...,    871,    545,      1],
       ...,
       [     0,      0,      0, ...,     64,   4291,      1],
       [     0,      0,      0, ...,   1168, 250412,      1],
       [     0,      0,      0, ...,  14192,    833,      1]], dtype=int32)

In [ ]:
embed = nn.Embedding(max_features, embed_size)

In [ ]:
embed.weight.data.copy_(torch.from_numpy(emb_matrix))

tensor([[-0.0539,  0.1281,  0.0639,  ..., -0.0137,  0.0337,  0.0956],
        [ 0.1327, -0.0386, -0.0923,  ...,  0.0126, -0.0587,  0.0935],
        [-0.0495,  0.2286,  0.0710,  ...,  0.0144,  0.1571,  0.0885],
        ...,
        [ 0.5255, -0.0496, -0.2679,  ..., -0.4167,  0.0296, -0.3673],
        [-0.2773,  0.0883, -0.0055,  ...,  0.3646,  0.1340, -0.1685],
        [ 0.0186,  0.2157,  0.1814,  ..., -0.1296, -0.0219,  0.2497]])

In [ ]:
embed(torch.from_numpy(train_X[0]))

tensor([[-0.0539,  0.1281,  0.0639,  ..., -0.0137,  0.0337,  0.0956],
        [-0.0539,  0.1281,  0.0639,  ..., -0.0137,  0.0337,  0.0956],
        [-0.0539,  0.1281,  0.0639,  ..., -0.0137,  0.0337,  0.0956],
        ...,
        [-0.0617,  0.2367, -0.2441,  ..., -0.0792,  0.2135,  0.0753],
        [-0.1293, -0.1769, -0.1770,  ...,  0.0189,  0.0535, -0.0185],
        [ 0.1327, -0.0386, -0.0923,  ...,  0.0126, -0.0587,  0.0935]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, emb_matrix):
        super(LSTM, self).__init__()

        embedding = nn.Embedding(max_features, embed_size)
        embedding.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.embedding = embedding
        self.lstm = nn.LSTM(input_size=300, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(128*2, 1)
        self.logit = nn.Linear(1, 1)
      
    def forward(self, x):
        # x = self.embedding(x)
        # out, (hidden, _) = self.lstm(x)
        # x = self.fc(x)
        # x = self.logit(x).view(-1)
        x = self.embedding(x)

        return x

In [ ]:
model = LSTM(emb_matrix).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
max_epoch = 2

In [ ]:
num_data = train_X.shape[0]

In [ ]:
train_y.shape

(940407,)

In [ ]:
X_train = torch.tensor(train_X, dtype=torch.long).cuda()
y_train = torch.tensor(train_y[:, np.newaxis], dtype=torch.float32).cuda()

In [ ]:
train = torch.utils.data.TensorDataset(X_train, y_train)

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=512)

In [ ]:
k = 1
for i in train_loader:
    k += 1
    if k == 3:
        print(i[1])
        break

In [ ]:
for epoch in range(max_epoch):
    model.train()
    for inputs, labels in train_loader:

        # inputs = torch.from_numpy(inputs).to(device)
        # labels = torch.from_numpy(labels)
        # labels = labels.to(device)

        logits = model(inputs)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()



ValueError: ignored

In [ ]:
train_X

tensor([[     0,      0,      0,  ...,     77,  11994,      1],
        [     0,      0,      0,  ...,   3422,  23617,      1],
        [     0,      0,      0,  ...,    871,    545,      1],
        ...,
        [     0,      0,      0,  ...,     64,   4291,      1],
        [     0,      0,      0,  ...,   1168, 250412,      1],
        [     0,      0,      0,  ...,  14192,    833,      1]],
       device='cuda:0', dtype=torch.int32)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

https://www.kaggle.com/theoviel/quora-final-submission